## IMPORT

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import *

from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
import math

from joblib import dump, load

lookup_df = pd.read_csv("zone_lookup.csv")

DATEPARSE = "%Y-%m-%d %H:%M:%S"

## Clean data

In [ ]:
def time_to_minutes(_today):
    return (_today.hour*60) + (_today.minute)
def add_zero_padding(number):
    if len(str(number)) == 1: return "0" + str(number)
    else: return number
def format_df(DF, pickup_date_column, dropof_date_column, month):
    #Clean some rows where the location id is invalid
    DF = DF.dropna()
    DF = DF[((DF["PULocationID"] != 264) & (DF["PULocationID"] != 265)) & ((DF["DOLocationID"] != 264) & (DF["DOLocationID"] != 265))]

    start = pd.to_datetime(DF[pickup_date_column])
    stop = pd.to_datetime(DF[dropof_date_column])
    DF['travel_time'] = (stop - start).astype('timedelta64[m]').astype('int64')

    DF = DF[(DF["travel_time"] < 5*60)]

    DF['PU_start'] = (start).dt.hour

    DF['PUlat'] = DF.PULocationID.map(lookup_df.set_index('LocationID').lat)
    DF['PUlong'] = DF.PULocationID.map(lookup_df.set_index('LocationID').long)
    DF['DOlat'] = DF.DOLocationID.map(lookup_df.set_index('LocationID').lat)
    DF['DOlong'] = DF.DOLocationID.map(lookup_df.set_index('LocationID').long)

    DF['distance'] = ((DF.PUlat - DF.DOlat) ** 2 + (DF.PUlong - DF.DOlong) ** 2) ** 0.5

    DF = DF.drop(columns=[dropof_date_column, pickup_date_column])
    return DF

In [ ]:
#dataset = ("taxi_data/green_tripdata_2019", "l")
dataset = ("taxi_data/yellow_tripdata_2019", "t")

pd.options.mode.chained_assignment = None
header = True
for x in range(1, 13): # this takes 5 min
    try:
        print("Reading csv: " + str(x))
        dfGreen = pd.read_csv(dataset[0] + "-" + str(add_zero_padding(x)) + ".csv", usecols=["PULocationID", "DOLocationID", dataset[1] + "pep_pickup_datetime", dataset[1] + "pep_dropoff_datetime"])
        print("Formating csv")
        dfGreen = format_df(dfGreen, dataset[1] + "pep_pickup_datetime", dataset[1] + "pep_dropoff_datetime", x)
        print("Writing csv")
        dfGreen.to_csv(dataset[0] + "_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("DONE: " + str(x))
    except Exception as f:
        print(f)
pd.options.mode.chained_assignment = "warn"